In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Ensure the Ollama server is running and the model is pulled
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec

In [16]:
import json
import re
import pdfplumber
import docx
import gradio as gr
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Initialize OpenAI client for Ollama
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Function to extract text from PDF or DOCX
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# Llama 3.2 prompt for structured extraction
EXTRACT_FEATURES_FROM_RESUME = """
You are an AI assistant that extracts structured JSON from resumes and matches them with job descriptions.

Expected JSON format:

{
  "personal": {
    "name": "Full Name",
    "phone": "Phone Number",
    "email": "Email Address",
    "address": "Address",
    "gender": "Male/Female/Other",
    "date_of_birth": "YYYY-MM-DD",
    "social": ["LinkedIn", "GitHub"],
    "about": "Short personal summary"
  },
  "skills": [
    {"name": "Skill Name", "type": "Programming/Soft Skill", "experience": 2}
  ],
  "education": [
    {"degree": "Degree Name", "institution": "University Name"}
  ],
  "experience": [
    {
      "title": "Job Title",
      "company": "Company Name",
      "dates": "Start - End Date",
      "location": "Location",
      "responsibilities": ["Responsibility 1", "Responsibility 2"],
      "experience_years": 2
    }
  ],
  "overall_experience": 2,
  "salary": 75000.0
}
"""

# Function to query Llama 3.2
def query_llama_api(resume_text, job_description):
    try:
        prompt = f"Resume Text: {resume_text}\nJob Description: {job_description}\n\n{EXTRACT_FEATURES_FROM_RESUME}"

        response = ollama_via_openai.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=600,
            temperature=0.7,
        )

        raw_output = response.choices[0].text.strip()

        # Extract JSON from response using regex
        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        else:
            return {"error": "Invalid JSON extracted from response"}

    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON from API response"}
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

# Function to calculate match score
def calculate_match_score(resume_text, jd_text):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([resume_text, jd_text])
    similarity_score = cosine_similarity(vectors)[0][1] * 100  # Convert to percentage
    return round(similarity_score, 2)

# Function to process the resume with Gradio UI
def process_resume(file, job_role):
    try:
        file_path = file.name
        resume_text = extract_text_from_resume(file_path)

        # Pre-defined job descriptions (JD)
        job_descriptions = {
            "Software Engineer": "Looking for a software engineer with Python, Java, and cloud experience.",
            "Data Scientist": "Seeking a data scientist proficient in machine learning, deep learning, and data visualization.",
            "AI Engineer": "Hiring an AI Engineer with expertise in neural networks, NLP, and computer vision.",
            "Cybersecurity Analyst": "Looking for a cybersecurity analyst skilled in ethical hacking and threat detection."
        }

        job_description = job_descriptions.get(job_role, "No JD available.")

        extracted_data = query_llama_api(resume_text, job_description)

        # Calculate match score
        match_score = calculate_match_score(resume_text, job_description)
        extracted_data["match_score"] = match_score  # Add score to JSON output

        return json.dumps(extracted_data, indent=4)

    except Exception as e:
        return json.dumps({"error": str(e)})

# Gradio Interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF/DOCX)"),
        gr.Dropdown(
            ["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"],
            label="Select Job Role",
            value="Software Engineer"
        )
    ],
    outputs=gr.JSON(label="Extracted Resume Data & Match Percentage"),
    title="Resume Extractor & JD Matcher with Llama 3.2",
    description="Upload a resume and select a job role to extract structured information & match percentage.",
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [18]:
import json
import re
import pdfplumber
import docx
import gradio as gr
from openai import OpenAI

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Initialize OpenAI client for Ollama
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Function to extract text from PDF or DOCX
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# Function to calculate match score & missing elements
def calculate_match(extracted_data, job_description):
    required_skills = {
        "Software Engineer": ["Python", "Java", "Cloud Computing"],
        "Data Scientist": ["Machine Learning", "Deep Learning", "Data Visualization"],
        "AI Engineer": ["Neural Networks", "NLP", "Computer Vision"],
        "Cybersecurity Analyst": ["Ethical Hacking", "Threat Detection"]
    }
    
    # Get expected skills for job role
    job_role = job_description.split("with ")[-1].split(" experience")[0]
    expected_skills = required_skills.get(job_role, [])
    
    # Extract skills from resume
    extracted_skills = [skill["name"] for skill in extracted_data.get("skills", [])]
    
    matched_skills = [skill for skill in expected_skills if skill in extracted_skills]
    missing_skills = [skill for skill in expected_skills if skill not in extracted_skills]
    
    # Check missing personal info
    missing_info = []
    for key in ["gender", "date_of_birth"]:
        if not extracted_data["personal"].get(key):
            missing_info.append(key)
    
    # Calculate match score
    skill_score = (len(matched_skills) / len(expected_skills)) * 50 if expected_skills else 0
    experience_score = min(50, extracted_data.get("overall_experience", 0) * 10)
    match_score = round(skill_score + experience_score, 2)
    
    extracted_data.update({
        "match_score": match_score,
        "matched_skills": matched_skills,
        "missing_skills": missing_skills,
        "missing_info": missing_info,
        "match_breakdown": {
            "skill_score": skill_score,
            "experience_score": experience_score
        }
    })
    
    return extracted_data

# Function to query Llama 3.2
def query_llama_api(resume_text, job_description):
    try:
        prompt = f"Resume Text: {resume_text}\nJob Description: {job_description}\n\n{EXTRACT_FEATURES_FROM_RESUME}\nEnsure the output is strictly in JSON format without any additional text."

        response = ollama_via_openai.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=1000,  # Increase to capture full JSON
            temperature=0.7,
        )

        raw_output = response.choices[0].text.strip()
        print("Raw API Response:", raw_output)  # Debugging step

        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            return json.loads(match.group(0))
        else:
            return {"error": "Invalid JSON extracted from response"}

    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON from API response"}
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}


# Function to process the resume
def process_resume(file, job_role):
    try:
        file_path = file.name
        resume_text = extract_text_from_resume(file_path)

        job_descriptions = {
            "Software Engineer": "Looking for a software engineer with Python, Java, and cloud experience.",
            "Data Scientist": "Seeking a data scientist proficient in machine learning, deep learning, and data visualization.",
            "AI Engineer": "Hiring an AI Engineer with expertise in neural networks, NLP, and computer vision.",
            "Cybersecurity Analyst": "Looking for a cybersecurity analyst skilled in ethical hacking and threat detection."
        }

        job_description = job_descriptions.get(job_role, "No JD available.")
        extracted_data = query_llama_api(resume_text, job_description)

        return json.dumps(extracted_data, indent=4)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF/DOCX)"),
        gr.Dropdown(
            choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"],
            label="Select Job Role"
        )
    ],
    outputs=gr.JSON(label="Extracted Resume Data & Match Analysis"),
    title="Resume Analyzer & JD Matcher",
    description="Upload a resume and select a job role to extract structured information, match percentage, and missing information.",
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


Raw API Response: {
  "personal": {
    "name": "Venkata Surya Vishal Ganti",
    "phone": "+91 8500031155",
    "email": "suryavishal2002@gmail.com",
    "address": "Srikakulam, Andhra Pradesh 532005",
    "gender": "Male",
    "date_of_birth": "Not Available",
    "social": ["LinkedIn", "https://www.linkedin.com/in/surya-vishal-ganti/"],
    "about": "Highly motivated Graduate Apprentice Developer with a strong foundation in computer vision, machine learning, and deep learning."
  },
  "skills": [
    {"name": "Python", "type": "Programming", "experience": 2},
    {"name": "SQL", "type": "Programming", "experience": 1},
    {"name": "Pandas", "type": "Programming", "experience": 1},
    {"name": "NumPy", "type": "Programming", "experience": 1},
    {"name": "Scikit-Learn", "type": "Programming", "experience": 2},
    {"name": "Matplotlib", "type": "Programming", "experience": 1},
    {"name": "OpenCV", "type": "Programming", "experience": 2},
    {"name": "TensorFlow", "type": "Progr

In [23]:
import json
import re
import pdfplumber
import docx
import gradio as gr
from openai import OpenAI
from datetime import datetime

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Prompt for feature extraction
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0  # Total years of experience
}

Rules:
1. Ensure all keys are present, use empty strings/lists if not found
2. Categorize skills as 'technical' or 'soft'
3. Calculate overall experience from work experience entries
4. Extract all relevant details from the resume
5. Provide response in strict JSON format
"""

# Initialize OpenAI client for Ollama
ollama_via_openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Function to parse experience duration
def parse_experience_duration(duration_str):
    """
    Parse work experience duration from various string formats
    
    Handles formats like:
    - "2 years"
    - "1.5 years"
    - "November 2020 - Present"
    - "June 2018 - August 2022"
    """
    # Direct numeric years extraction
    years_match = re.search(r'(\d+(?:\.\d+)?)\s*years?', str(duration_str), re.IGNORECASE)
    if years_match:
        return float(years_match.group(1))
    
    # Date range parsing
    date_range_match = re.findall(r'(\w+ \d{4})', str(duration_str))
    if len(date_range_match) == 2:
        try:
            start_date = datetime.strptime(date_range_match[0], "%B %Y")
            end_date = datetime.strptime(date_range_match[1], "%B %Y") if date_range_match[1] != "Present" else datetime.now()
            
            # Calculate years of experience
            years = (end_date.year - start_date.year) + (end_date.month - start_date.month) / 12
            return round(years, 2)
        except ValueError:
            pass
    
    # Fallback: return 0 if no parseable duration found
    return 0

# Function to extract text from PDF or DOCX
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text.strip()

# Enhanced Function to calculate match score & provide detailed insights
def calculate_match(extracted_data, job_description):
    # Comprehensive skill requirements for different job roles
    required_skills = {
        "Software Engineer": {
            "technical": ["Python", "Java", "Cloud Computing", "Docker", "Kubernetes", "CI/CD", "Git"],
            "soft_skills": ["Problem Solving", "Teamwork", "Communication"],
            "required_info": ["email", "phone", "linkedin"]
        },
        "Data Scientist": {
            "technical": ["Machine Learning", "Deep Learning", "Data Visualization", "Python", "R", "SQL", "Pandas", "NumPy", "Scikit-learn"],
            "soft_skills": ["Analytical Thinking", "Communication", "Storytelling"],
            "required_info": ["email", "phone", "github"]
        },
        "AI Engineer": {
            "technical": ["Neural Networks", "NLP", "Computer Vision", "TensorFlow", "PyTorch", "Deep Learning", "Python"],
            "soft_skills": ["Research Skills", "Innovation", "Problem Solving"],
            "required_info": ["email", "phone", "research_publications"]
        },
        "Cybersecurity Analyst": {
            "technical": ["Ethical Hacking", "Threat Detection", "Network Security", "Penetration Testing", "SIEM", "Firewall Configuration"],
            "soft_skills": ["Attention to Detail", "Analytical Thinking", "Risk Assessment"],
            "required_info": ["email", "phone", "security_certifications"]
        }
    }
    
    # Get job role from job description
    job_role = job_description.split("with ")[-1].split(" experience")[0]
    job_skills = required_skills.get(job_role, {})
    
    # Extract skills from resume
    extracted_skills = {
        "technical_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "technical"],
        "soft_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "soft"]
    }
    
    # Skill Matching
    matched_technical_skills = [skill for skill in job_skills.get("technical", []) if skill in extracted_skills["technical_skills"]]
    missing_technical_skills = [skill for skill in job_skills.get("technical", []) if skill not in extracted_skills["technical_skills"]]
    
    matched_soft_skills = [skill for skill in job_skills.get("soft_skills", []) if skill in extracted_skills["soft_skills"]]
    missing_soft_skills = [skill for skill in job_skills.get("soft_skills", []) if skill not in extracted_skills["soft_skills"]]
    
    # Missing Personal Information
    missing_personal_info = [info for info in job_skills.get("required_info", []) 
                              if not extracted_data.get("personal", {}).get(info)]
    
    # Match Score Calculation
    technical_skill_weight = 0.4  # 40% importance for technical skills
    soft_skill_weight = 0.2       # 20% importance for soft skills
    experience_weight = 0.2       # 20% importance for experience
    info_completeness_weight = 0.2  # 20% importance for personal info completeness
    
    # Technical Skills Score
    technical_skill_score = min(1.0, len(matched_technical_skills) / len(job_skills.get("technical", [1]))) * technical_skill_weight * 100
    
    # Soft Skills Score
    soft_skill_score = min(1.0, len(matched_soft_skills) / len(job_skills.get("soft_skills", [1]))) * soft_skill_weight * 100
    
    # Experience Score
    experience_score = min(50, extracted_data.get("overall_experience", 0) * 10) * experience_weight
    
    # Information Completeness Score
    info_completeness_score = (1 - (len(missing_personal_info) / len(job_skills.get("required_info", [1])))) * info_completeness_weight * 100
    
    # Total Match Score
    match_score = round(technical_skill_score + soft_skill_score + experience_score + info_completeness_score, 2)
    
    # Detailed Match Analysis
    match_breakdown = {
        "technical_skill_score": round(technical_skill_score, 2),
        "soft_skill_score": round(soft_skill_score, 2),
        "experience_score": round(experience_score, 2),
        "info_completeness_score": round(info_completeness_score, 2)
    }
    
    # Update extracted data with match details
    extracted_data.update({
        "match_score": match_score,
        "match_breakdown": match_breakdown,
        "skills_analysis": {
            "matched_technical_skills": matched_technical_skills,
            "missing_technical_skills": missing_technical_skills,
            "matched_soft_skills": matched_soft_skills,
            "missing_soft_skills": missing_soft_skills
        },
        "missing_personal_info": missing_personal_info
    })
    
    return extracted_data

# Function to query Llama 3.2
def query_llama_api(resume_text, job_description):
    try:
        prompt = f"Resume Text: {resume_text}\nJob Description: {job_description}\n\n{EXTRACT_FEATURES_FROM_RESUME}\nEnsure the output is strictly in JSON format without any additional text."

        response = ollama_via_openai.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=1000,  # Increase to capture full JSON
            temperature=0.7,
        )

        raw_output = response.choices[0].text.strip()
        print("Raw API Response:", raw_output)  # Debugging step

        match = re.search(r"\{.*\}", raw_output, re.DOTALL)
        if match:
            parsed_data = json.loads(match.group(0))
            
            # Calculate overall experience using enhanced parsing
            if parsed_data.get("work_experience"):
                experience_years = sum(
                    parse_experience_duration(exp.get("duration", "0")) 
                    for exp in parsed_data["work_experience"] 
                    if exp.get("duration")
                )
                parsed_data["overall_experience"] = round(experience_years, 2)
            else:
                parsed_data["overall_experience"] = 0
            
            return parsed_data
        else:
            return {"error": "Invalid JSON extracted from response"}

    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON from API response"}
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

# Function to process the resume
def process_resume(file, job_role):
    try:
        file_path = file.name
        resume_text = extract_text_from_resume(file_path)

        job_descriptions = {
            "Software Engineer": "Looking for a software engineer with Python, Java, and cloud experience.",
            "Data Scientist": "Seeking a data scientist proficient in machine learning, deep learning, and data visualization.",
            "AI Engineer": "Hiring an AI Engineer with expertise in neural networks, NLP, and computer vision.",
            "Cybersecurity Analyst": "Looking for a cybersecurity analyst skilled in ethical hacking and threat detection."
        }

        job_description = job_descriptions.get(job_role, "No JD available.")
        extracted_data = query_llama_api(resume_text, job_description)
        
        # Only calculate match if extraction was successful
        if not extracted_data.get("error"):
            matched_data = calculate_match(extracted_data, job_description)
            return json.dumps(matched_data, indent=4)
        else:
            return json.dumps(extracted_data, indent=4)
    
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
iface = gr.Interface(
    fn=process_resume,
    inputs=[
        gr.File(label="Upload Resume (PDF/DOCX)"),
        gr.Dropdown(
            choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"],
            label="Select Job Role"
        )
    ],
    outputs=gr.JSON(label="Extracted Resume Data & Match Analysis"),
    title="Resume Analyzer & JD Matcher",
    description="Upload a resume and select a job role to extract structured information, match percentage, and missing information.",
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


Raw API Response: {
    "personal": {
        "name": "",
        "email": "suryavishal2002@gmail.com",
        "phone": "+91 8500031155",
        "linkedin": "https://www.linkedin.com/in/surya-vishal-ganti/",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical",
            "proficiency": ""
        },
        {
            "name": "",
            "type": "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "B. Tech",
            "institution": "Anil Neerukonda Institute of Technology and Sciences(ANITS) Tagarapuvalasa, Vizag",
            "year": "November 2021 - April 2024"
        },
        {
            "degree": "Diploma",
            "institution": "Government Polytechnic Srikakulam, Srikakulam",
            "year": "July 2018 - October 2021"
        }
    ],
    "work_experience": [
        {
            "company": "Tekworks Enterpr

In [24]:
import json
import re
import pdfplumber
import docx
import gradio as gr
from datetime import datetime

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Extract features from resume
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0
}
"""

# Function to extract text from resume
def extract_text_from_resume(file_path):
    text = ""
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs])
    return text.strip()

# Function to calculate match
def calculate_match(extracted_data, job_role):
    job_requirements = {
        "Software Engineer": {
            "technical_skills": ["Python", "Java", "Cloud Computing", "Docker", "Kubernetes", "CI/CD", "Git", "SQL", "REST APIs"],
            "soft_skills": ["Problem Solving", "Teamwork", "Communication", "Agile Methodology", "Critical Thinking"],
            "required_info": ["email", "phone", "linkedin"],
            "min_experience": 2
        },
        "Data Scientist": {
            "technical_skills": ["Machine Learning", "Deep Learning", "Data Visualization", "Python", "R", "SQL", "Pandas", "NumPy", "Scikit-learn", "TensorFlow", "Keras"],
            "soft_skills": ["Analytical Thinking", "Communication", "Storytelling", "Problem Solving", "Research Skills"],
            "required_info": ["email", "phone", "github"],
            "min_experience": 1.5
        },
        "AI Engineer": {
            "technical_skills": ["Neural Networks", "NLP", "Computer Vision", "TensorFlow", "PyTorch", "Deep Learning", "Python", "Machine Learning", "Keras"],
            "soft_skills": ["Research Skills", "Innovation", "Problem Solving", "Analytical Thinking", "Creativity"],
            "required_info": ["email", "phone", "research_publications"],
            "min_experience": 2
        },
        "Cybersecurity Analyst": {
            "technical_skills": ["Ethical Hacking", "Threat Detection", "Network Security", "Penetration Testing", "SIEM", "Firewall Configuration", "Vulnerability Assessment", "Security Protocols"],
            "soft_skills": ["Attention to Detail", "Analytical Thinking", "Risk Assessment", "Problem Solving", "Communication"],
            "required_info": ["email", "phone"],
            "min_experience": 1
        }
    }

    # Get job role requirements
    requirements = job_requirements.get(job_role, {})
    
    # Extract candidate skills
    candidate_skills = {
        "technical_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "technical"],
        "soft_skills": [skill["name"] for skill in extracted_data.get("skills", []) if skill.get("type") == "soft"]
    }
    
    # Match skills
    matched_technical = [skill for skill in requirements.get("technical_skills", []) if skill in candidate_skills["technical_skills"]]
    missing_technical = [skill for skill in requirements.get("technical_skills", []) if skill not in candidate_skills["technical_skills"]]
    matched_soft = [skill for skill in requirements.get("soft_skills", []) if skill in candidate_skills["soft_skills"]]
    missing_soft = [skill for skill in requirements.get("soft_skills", []) if skill not in candidate_skills["soft_skills"]]
    
    # Missing personal info
    missing_info = [info for info in requirements.get("required_info", []) if not extracted_data.get("personal", {}).get(info)]
    
    # Experience check
    experience = extracted_data.get("overall_experience", 0)
    meets_experience = experience >= requirements.get("min_experience", 0)
    
    # Match Score Calculation
    technical_weight = 0.4
    soft_skills_weight = 0.2
    experience_weight = 0.2
    info_weight = 0.2
    
    technical_score = (len(matched_technical) / max(len(requirements.get("technical_skills", [])), 1)) * 100 * technical_weight
    soft_skills_score = (len(matched_soft) / max(len(requirements.get("soft_skills", [])), 1)) * 100 * soft_skills_weight
    experience_score = (experience / max(requirements.get("min_experience", 1), 1)) * 100 * experience_weight
    info_score = ((len(requirements.get("required_info", [])) - len(missing_info)) / max(len(requirements.get("required_info", [])), 1)) * 100 * info_weight
    
    overall_score = round(technical_score + soft_skills_score + experience_score + info_score, 2)
    
    # Result
    return {
        "Job Role": job_role,
        "Overall Match Score": f"{overall_score}%",
        "Matched Technical Skills": matched_technical,
        "Missing Technical Skills": missing_technical,
        "Matched Soft Skills": matched_soft,
        "Missing Soft Skills": missing_soft,
        "Experience": f"{experience} years (Required: {requirements.get('min_experience', 0)} years)",
        "Missing Personal Info": missing_info
    }

# Function to process resume and compute match
def process_resume(file, job_role):
    if not file:
        return "Please upload a resume."

    file_path = file.name
    text = extract_text_from_resume(file_path)

    if not text:
        return "Could not extract text from the resume."

    # Simulated function for now (replace with Llama API call later)
    extracted_data = {
        "personal": {"name": "John Doe", "email": "john.doe@example.com", "phone": "1234567890"},
        "skills": [{"name": "Python", "type": "technical"}, {"name": "Communication", "type": "soft"}],
        "education": [{"degree": "BSc Computer Science", "institution": "XYZ University", "year": "2020"}],
        "work_experience": [{"company": "ABC Corp", "role": "Software Developer", "duration": "2 years"}],
        "overall_experience": 2
    }

    return calculate_match(extracted_data, job_role)

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Resume Matching System")
    file_input = gr.File(label="Upload Resume (PDF/DOCX)")
    job_role_input = gr.Dropdown(choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"], label="Select Job Role")
    submit_button = gr.Button("Analyze Resume")
    output = gr.JSON(label="Match Results")

    submit_button.click(process_resume, inputs=[file_input, job_role_input], outputs=output)

# Launch the Gradio app
demo.launch()


* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


In [33]:
import json
import re
import pdfplumber
import docx
import gradio as gr
import requests
from datetime import datetime

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

# Extract features from resume
EXTRACT_FEATURES_FROM_RESUME = """
Extract the following structured information from the resume:
{
    "personal": {
        "name": "",
        "email": "",
        "phone": "",
        "linkedin": "",
        "github": "",
        "research_publications": []
    },
    "skills": [
        {
            "name": "",
            "type": "technical" or "soft",
            "proficiency": ""
        }
    ],
    "education": [
        {
            "degree": "",
            "institution": "",
            "year": ""
        }
    ],
    "work_experience": [
        {
            "company": "",
            "role": "",
            "duration": ""
        }
    ],
    "overall_experience": 0
}
"""

# Function to extract text from resume
def extract_text_from_resume(file_path):
    text = ""
    try:
        if file_path.endswith(".pdf"):
            with pdfplumber.open(file_path) as pdf:
                text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
        elif file_path.endswith(".docx"):
            doc = docx.Document(file_path)
            text = "\n".join([para.text for para in doc.paragraphs])
        else:
            return None  # Return None if the file type is unsupported
    except Exception as e:
        return f"Error extracting text: {str(e)}"
    
    return text.strip()

# Function to call Llama API for text extraction
def extract_data_with_llama(text):
    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": EXTRACT_FEATURES_FROM_RESUME + "\n" + text}]
    }
    try:
        response = requests.post(OLLAMA_API, json=payload)
        response.raise_for_status()
        
        # Print the raw response for debugging
        print("Raw Response:", response.text)
        
        response_json = response.json()
        content = response_json.get('message', {}).get('content', '')
        
        # Try to extract JSON from the content
        # Look for text between first { and last }
        import re
        json_match = re.search(r'\{.*\}', content, re.DOTALL)
        if json_match:
            content = json_match.group(0)
        
        print("Extracted Content:", content)
        
        # Try parsing the content as JSON
        try:
            parsed_json = json.loads(content)
            return json.dumps(parsed_json)  # Return a clean JSON string
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {e}")
            return "Failed to parse JSON: " + content
    
    except requests.exceptions.RequestException as e:
        return f"Request error: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

# Rest of the code remains the same as in the original script
# (calculate_match, process_resume, Gradio interface setup)

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Resume Matching System")
    file_input = gr.File(label="Upload Resume (PDF/DOCX)")
    job_role_input = gr.Dropdown(choices=["Software Engineer", "Data Scientist", "AI Engineer", "Cybersecurity Analyst"], label="Select Job Role")
    submit_button = gr.Button("Analyze Resume")
    output = gr.JSON(label="Match Results")

    submit_button.click(process_resume, inputs=[file_input, job_role_input], outputs=output)

# Launch the Gradio app
demo.launch()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:httpcore.connection:connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=3 socket_options=None
DEBUG:httpcore.connection:connect_tcp.started host='127.0.0.1' port=7895 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x3405dfa90>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'date', b'Tue, 25 Mar 2025 06:37:22 GMT'), (b'server', b'uvicorn'), (b'content-length', b'4'), (b'content-type', 

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x340350450>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x339d1ede0> server_hostname='api.gradio.app' timeout=3
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /api/telemetry/gradio/initiated HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /api/telemetry/gradio/launched HTTP/1.1" 200 0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x32caa0b90>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET'